In [1]:
import time, os, json, PyPDF2, re
import pandas as pd

# PDF -> Text

In [2]:
def read_pdf2012(pdf_path, r, name):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = '\n'.join([i.extract_text().replace('\t', ' ') for i in reader.pages])
    r[name] = text

In [3]:
pdf_directory = '/Users/bjcliang/Desktop/Nate/Census/Surveys2012'
record = dict()

for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        print(pdf_path, end = '\r')
        read_pdf2012(pdf_path, record, pdf_file)
with open('/Users/bjcliang/Desktop/Nate/Census/Surveys2012/rec.json', 'w') as json_file:
    json.dump(record, json_file)

# Clean Data

In [1]:
import time, os, json, PyPDF2, re, docx
import pandas as pd
from docx.oxml.ns import qn
from docx.shared import Pt

In [2]:
def general_process(text):
    def find_variations_with_spaces(sp, text):
        sp = sp.strip().split(' ')
        pattern = r'\s*'.join(sp)
        matches = re.findall(pattern, text)
        return matches

    # Pattern to match: 'Form' followed by letters, dashes, numbers, and a date in parentheses
    text = re.sub('\n', ' ', text)
    pattern = r'Form [A-Z0-9\-]+ \(\d{1,2}-\d{1,2}-\d{4}\)'
    text = re.sub(pattern, '', text)
    text = ' '.join(text.split())
    pattern = r'CONTINUE WITH[\w\s\-]+- Continued'
    text = re.sub(pattern, '', text)
    pattern = r'CONTINUE ON PAGE[\d\s]+'
    text = re.sub(pattern, '', text)
    pattern = r'Page[\d\s]+'
    text = re.sub(pattern, '', text)
    text = re.sub('INFORMATION COPY DO NOT USE TO REPORT', '', text)
    text = text.split('(Mark "X" only ONE box.')[0]
    text = text.split('Mark "X" only ONE box.')[0]
    text = re.sub(r'(?:\. ){3,}', ' ', text)
    
    text = text.replace('HOW TO REPORTPERCENTS', '\nHOW TO REPORTPERCENTS')
    text = re.sub(r'CONTINUE WITH \d{1,2} ON PAGE \d{1,2}', ' ', text)
    text = re.sub('Line', '\nLine', text)

    text = re.sub(r' (\(\d+\))', r'\n\1', text)
    text = re.sub(r'(\s\d+\.\s)', r'\n\1', text)
    text = re.sub(r'(\s[a-z]\. )', r'\n\1', text)
     
    
    pattern = r'\b\d+\s*(?:-\s*|\band\b\s*)\s*\d+\s*Not Applicable\.\s*'
    text = re.sub(pattern, '', text)
    text = re.sub(r'\b\d+\s+Not Applicable\.\s*', '', text)
    text = re.sub(r'\b\d{8}\b', '', text)
    text = re.sub('A. ', '', text)
    #text = re.sub('-', '', text)
    text = re.sub('8 –', '', text)
    text = re.sub('Internal RevenueService', 'Internal Revenue Service', text)
    
    text = text.replace('If not shown, please enter your 11-digit Census File Number (CFN) from the mailing address.', ' ')
    text = text.replace('Description of sales, shipments, receipts, or revenue 2012 Report thousands of dollars OR whole percents. Estimates are acceptable. $ Bil. Mil. Thou. Percent', '')
    
    text = '\n'.join([i for i in text.split('\n') if len(i.strip())>3])
    text = ' '.join([i for i in text.split(' ') if len(i)>0])
    text = re.sub(r'\s+\d+$', '', text.strip())
    
    sp = 'HOW TO REPORT DOLLAR FIGURES'
    for ff in find_variations_with_spaces(sp, text):
        text = text.replace(ff, sp)
    
    return text.strip()

In [3]:
def find_variations_with_spaces(sp, text):
        sp = sp.strip().split(' ')
        pattern = r'\s*'.join(sp)
        matches = re.findall(pattern, text)
        return matches

In [4]:
def find_splitters(text):
    #pattern = r'\d{1,2}\n [A-Z ,0-9]+\n'
    target = re.findall(r'\d{1,2}\n [A-Z, ]+\n', text)
    target = target + re.findall(r'\b\d{1,2}\n [A-Z, -]+(?: - Continued)?\n', text)
    #pattern = r'\d+\n[A-Z ]+\n'
    #target = target + re.findall(pattern, text) 
    tar = []
    for i in target:
        check = True
        for k in ['DUE', 'HOW TO', 'INFORMATION COPY', '2012', '\n ON\n', 
                  'ECONOMIC', 'EXAMPLE', 'DAYS', 'CONTINUE', 'Continued']:
            if k in i:
                check = False
        #if not bool(re.match(r'^[A-Za-z\s]+$', i[3:])): check = False
        if check:
            n = i.replace('\n', '')
            n = ' '.join([k for k in n.split(' ') if len(k)>1])
            if len(n)>5:
                tar.append(i)
                
    if '30\n CERTIFICATION' in text: tar.append('30\n CERTIFICATION')
    
    return sorted(set(tar), key = lambda x:int(x[:2]))

In [5]:
record = json.load(open('/Users/bjcliang/Desktop/Nate/Census/Surveys2012/rec.json'))

In [6]:
rec_new = dict()
for survey in record:
    text = record[survey]
    splts = find_splitters(text)
    prev = text
    for sp in splts[::-1]:
        tt = text.split(sp)[-1]
        prev = prev[:-len(tt)]
        item = sp.replace('\n', '')
        item = ' '.join([i for i in item.split(' ') if len(i)>0])
        if item not in rec_new: rec_new[item] = dict()
        
        for other in [i.split('\n')[1].strip() for i in splts if i not in sp]:
            tt = tt.split(other)[0]
        
        tt = general_process(tt)
        
        if tt not in rec_new[item]: rec_new[item][tt] = []

        rec_new[item][tt].append(survey)

In [7]:
rec_new.keys()

dict_keys(['30 CERTIFICATION', '22 DETAIL OF SALES, SHIPMENTS, RECEIPTS, OR REVENUE', '19 KIND OF BUSINESS OR ACTIVITY', '7 EMPLOYMENT AND PAYROLL', '5 SALES, SHIPMENTS, RECEIPTS, OR REVENUE', '4 MONTHS IN OPERATION', '3 OPERATIONAL STATUS', '2 PHYSICAL LOCATION', '1 EMPLOYER IDENTIFICATION NUMBER', '26 SPECIAL INQUIRIES', '25 EXPORTED SERVICES', '20 CLASS OF CUSTOMER', '19 KIND OF BUSINESS', '22 DETAIL OF REVENUE OR RECEIPTS', '5 REVENUE OR RECEIPTS', '21 SUPPORT SERVICES', '25 EXPORTS OF GOODS AND SERVICES', '10 INVENTORIES BY VALUATION METHOD', '9 VALUE OF INVENTORIES', '16 SELECTED EXPENSES', '28 LOCATIONS OF OPERATION'])

In [59]:
def create_word_document_from_dict(data, path):
    doc = docx.Document()
    doc.add_heading('Economic Census Surveys 2012 (Services)', level=1)
    
    for superitem in sorted(data.keys(), key = lambda x: int(x[:2])):
        
        doc.add_heading("ITEM " + superitem, level=2)
        for item_index, (item, details) in enumerate(data[superitem].items(), start=1):
            doc.add_paragraph(item).style='List Bullet'
            
            surveys = ', '.join(details)
            p = doc.add_paragraph()
            p = p.add_run(f'Surveys included: {surveys}')
            p.font.size = Pt(9)
                #p.italic = True   

    # Save the document
    doc.save(path)

create_word_document_from_dict(rec_new, 'Economic Census Surveys 2012 new.docx')

In [136]:
def clean_specialq(text, names, r):
    
    if 'PROFESSIONEMPLOYER' in text:
    #if 'MN-55102.pdf' in names:
        print(names, text)
        print('------')
    nnames = [n for n in names]
    exl1 = '– REMARKS (Please use this space for any explanations that may be essential in understanding your reported data.) PLEASE PHOTOCOPY THIS FORM FOR YOUR RECORDS AND RETURN THE ORIGINAL. Thank you for completing your 2012 ECONOMIC CENSUS form. Date completed E-mail address Month Day Year Tele- phone -- - Fax -- Area code Number Extension Area code Number Name of person to contact regarding this report Title Is the time period covered by this report acalendar year? Yes No - Enter time period covered Month Year Month Year FROM TO'
    exl = '– REMARKS (Please use this space for any explanations that may be essential in understanding your reported data.) PLEASE PHOTOCOPY THIS FORM FOR YOUR RECORDS AND RETURN THE ORIGINAL. Thank you for completing your 2012 ECONOMIC CENSUS form. Datecompleted E-mail address Month Day Year Tele- phone -- - Fax -- Area code Number Extension Area code Number Name of person to contact regarding this report Title Is the time period covered by this report acalendar year? Yes No - Enter time period covered Month Year Month Year FROM TO'
    text = text.replace(exl, '')
    text = text.replace(exl1, '')
    text = text.replace('SPECIINQUIRIES - Continued', '')
    matches = [match.strip() for match in re.findall(r'[A-Z\s,/.]+', text) if match.strip()]
    matches = [re.sub(r'\s[A-Z]$', '', i) for i in matches]
    
    
    def clean_m(text):
        a = text.replace(',', ' ').replace('.', ' ')
        return ' '.join([i.strip() for i in a.split(' ') if len(i.strip())>1])
    
    matches = [(i, clean_m(i)) for i in matches if (len(i.strip())>1)]
    #print(matches)
    
    for d in ["INQUIRIES", 'TOT', 'IRS', 'NONE', 'DVD']:
        matches = [(i, j) for (i, j) in matches if (d not in j)]
    
    neg = ['ROM', 'NOT', 'PEO', 'LESS','LONG', 'DISTANCE', 
           'LOC', 'OR', 'ROM,', 'CD','/P', 'ONE', 'SEPARATE', 'UNDER CONTRACT']
    matches = [(i, j) for (i, j) in matches if (j not in neg)]
    
    def remove_single(text):
        return ' '.join([i for i in text.split('.') if len(i.strip())>1])
    
    matches = [(i, j) for (i, j) in matches if len(remove_single(j))>0]
   

    prev = text
    #r = []
    for org, m in matches[::-1]:
        tt = prev.split(org)[-1]
        prev = prev.split(org)[0]
        tt = tt.strip()
        tt = re.sub(r' \d+$', '', tt)
        #if tt == t: 
        #    print(names)
                    
        #m = ' '.join([i.strip() for i in m.replace(',', '').replace('.', '').split(' ') if len(i)>1])
        #r.append((m, tt))
        
        if m not in r:r[m] = dict()
        if tt not in r[m]: r[m][tt] = []
        r[m][tt].extend(nnames)
    #return r

In [119]:
doc = docx.Document()
for m in rec_special:
    doc.add_heading(m, level=3)
    #print(m)
    for tt in rec_special[m]:
        doc.add_paragraph(tt).style='List Bullet'
        surveys = ', '.join(rec_special[m][tt])
        p = doc.add_paragraph()
        p = p.add_run(f'Surveys included: {surveys}')
        p.font.size = Pt(9)
doc.save('Economic Census Surveys 2012 new.docx')